In [10]:
import itertools
from numpy.random import choice
import random
from google.colab import files
from PIL import Image
import os
import numpy as np
import math
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torchvision
import tqdm


## Part 1

#### Generation of text

In [ ]:
styles = {
    'прическа':[
        'нет волос',
        'длинные в пучок',
        'длинные волнистые',
        'длинные прямые',
        'короткая волнистые',
        'короткая прямые',
        'короткая курчавые'
    ],
    'цвет волос':[
        'черный',
        'блонд',
        'каштановый',
        'пастельный розовый',
        'рыжий',
        'серебристо серый',
    ],
    'аксесуар':[
        'нет очков',
        'круглые очки',
        'солнцезащитные очки',
    ],
    'одежда':[
        'худи',
        'комбинезон',
        'футболка с круглым вырезом',
        'футболка с V-вырезом',
    ],
    'цвет одежды':[
        'черный',
        'синий',
        'серый',
        'зеленый',
        'оранжевый',
        'розовый',
        'красный',
        'белый'
    ],
}

styles_count = {
    'прическа':[
        7,
        0,
        1,
        23,
        1,
        11,
        7
    ],
    'цвет волос':[
        7,
        6,
        2,
        3,
        8,
        24,
    ],
    'аксесуар':[
        11,
        22,
        17,
    ],
    'одежда':[
        7,
        18,
        19,
        6,
    ],
    'цвет одежды':[
        4,
        5,
        6,
        8,
        6,
        8,
        7,
        6
    ],
}

In [ ]:
for value in styles_count.values():
  for i in range(len(value)):
    value[i] += 1
styles_count.values()

dict_values([[8, 1, 2, 24, 2, 12, 8], [8, 7, 3, 4, 9, 25], [12, 23, 18], [8, 19, 20, 7], [5, 6, 7, 9, 7, 9, 8, 7]])

In [ ]:
for k in styles_count.keys():
    summ = 0
    for s in styles_count[k]:
        summ += s
    print(k, summ)

прическа 57
цвет волос 56
аксесуар 53
одежда 54
цвет одежды 58


In [ ]:
attr_prob = {'прическа':[], 'цвет волос':[], 'аксесуар':[], 'одежда':[],'цвет одежды':[]}
for value in styles_count.values():
  counter = 0
  temp_probs = []
  while counter <= len(value):
    for i in range(len(value)):
      prob = value[i] / sum([el for el in value]) #if el != value[i]])
      temp_probs.append(prob)
      counter += 1
    attr_prob[list(filter(lambda x: styles_count[x] == value, styles_count))[0]] = temp_probs

print(attr_prob)


{'прическа': [0.14035087719298245, 0.017543859649122806, 0.03508771929824561, 0.42105263157894735, 0.03508771929824561, 0.21052631578947367, 0.14035087719298245, 0.14035087719298245, 0.017543859649122806, 0.03508771929824561, 0.42105263157894735, 0.03508771929824561, 0.21052631578947367, 0.14035087719298245], 'цвет волос': [0.14285714285714285, 0.125, 0.05357142857142857, 0.07142857142857142, 0.16071428571428573, 0.44642857142857145, 0.14285714285714285, 0.125, 0.05357142857142857, 0.07142857142857142, 0.16071428571428573, 0.44642857142857145], 'аксесуар': [0.22641509433962265, 0.4339622641509434, 0.33962264150943394, 0.22641509433962265, 0.4339622641509434, 0.33962264150943394], 'одежда': [0.14814814814814814, 0.35185185185185186, 0.37037037037037035, 0.12962962962962962, 0.14814814814814814, 0.35185185185185186, 0.37037037037037035, 0.12962962962962962], 'цвет одежды': [0.08620689655172414, 0.10344827586206896, 0.1206896551724138, 0.15517241379310345, 0.1206896551724138, 0.1551724137

In [ ]:
lengths = [len(lst) for lst in styles.values()]
indices = itertools.product(*[range(l) for l in lengths])
result = {}

for index in indices:
    combo = []
    product = 1
    key_combo = []
    for key, lst, i in zip(styles.keys(), styles.values(), index):
        combo.append(lst[i])
        product *= attr_prob[key][i]
        key_combo.append(f"{key}: {lst[i]}")
    result[', '.join(key_combo)] = product

print(result)

{'прическа: нет волос, цвет волос: черный, аксесуар: нет очков, одежда: худи, цвет одежды: черный': 5.797766301824104e-05, 'прическа: нет волос, цвет волос: черный, аксесуар: нет очков, одежда: худи, цвет одежды: синий': 6.957319562188924e-05, 'прическа: нет волос, цвет волос: черный, аксесуар: нет очков, одежда: худи, цвет одежды: серый': 8.116872822553745e-05, 'прическа: нет волос, цвет волос: черный, аксесуар: нет очков, одежда: худи, цвет одежды: зеленый': 0.00010435979343283386, 'прическа: нет волос, цвет волос: черный, аксесуар: нет очков, одежда: худи, цвет одежды: оранжевый': 8.116872822553745e-05, 'прическа: нет волос, цвет волос: черный, аксесуар: нет очков, одежда: худи, цвет одежды: розовый': 0.00010435979343283386, 'прическа: нет волос, цвет волос: черный, аксесуар: нет очков, одежда: худи, цвет одежды: красный': 9.276426082918565e-05, 'прическа: нет волос, цвет волос: черный, аксесуар: нет очков, одежда: худи, цвет одежды: белый': 8.116872822553745e-05, 'прическа: нет вол

In [ ]:
sum(list(result.values()))

1.0000000000000042

In [ ]:
generated = choice(list(result.keys()), 1, p=list(result.values()))
generated_prob = result[generated[0].item()]
print(f"{generated[0]}, вероятность стиля: {generated_prob}")

NameError: name 'result' is not defined

In [ ]:
## function

def generate_style(styles, styles_count, attr_prob=None, styles_num = 1):
  if attr_prob is None:
    for value in styles_count.values():
      for i in range(len(value)):
        value[i] += 1
    attr_prob = {key: [] for key in styles}
  for value in styles_count.values():
    counter = 0
    temp_probs = []
    while counter <= len(value):
      for i in range(len(value)):
        prob = value[i] / sum([el for el in value])
        temp_probs.append(prob)
        counter += 1
      attr_prob[list(filter(lambda x: styles_count[x] == value, styles_count))[0]] = temp_probs
  lengths = [len(lst) for lst in styles.values()]
  indices = itertools.product(*[range(l) for l in lengths])
  result = {}
  for index in indices:
      combo = []
      product = 1
      key_combo = []
      for key, lst, i in zip(styles.keys(), styles.values(), index):
          combo.append(lst[i])
          product *= attr_prob[key][i]
          key_combo.append(f"{key}: {lst[i]}")
      result[', '.join(key_combo)] = product
  generated_dict = {}
  generated = choice(list(result.keys()), styles_num, p=list(result.values()))
  if styles_num==1:
    generated_dict[generated.item()] = result[generated[0].item()]
    return generated_dict
  else:
    generated_dict = {}
    for i in range(len(generated)):
      generated_dict[generated[i]] = result[generated[i].item()]
    return generated_dict


In [ ]:
generated = generate_style(styles, styles_count)
print(list(generated.keys())[0], list(generated.values())[0])

прическа: нет волос, цвет волос: черный, аксесуар: нет очков, одежда: комбинезон, цвет одежды: серый 0.00019806910275486475


In [ ]:
generated = generate_style(styles, styles_count, attr_prob=attr_prob, styles_num=3)
for i in range(len(generated)):
  print(list(generated.keys())[i], list(generated.values())[i])
  print()

прическа: длинные прямые, цвет волос: черный, аксесуар: солнцезащитные очки, одежда: комбинезон, цвет одежды: серый 0.000804126699218468

прическа: короткая курчавые, цвет волос: серебристо серый, аксесуар: нет очков, одежда: комбинезон, цвет одежды: белый 0.0005721996301807204

прическа: длинные прямые, цвет волос: рыжий, аксесуар: солнцезащитные очки, одежда: комбинезон, цвет одежды: зеленый 0.0011168426378034275



In [ ]:
## generator class
class StyleGenerator:

    def __init__(self, styles, styles_count, attr_prob=None, styles_num=1):
        self.styles = styles
        self.styles_count = styles_count
        self.attr_prob = attr_prob
        self.styles_num = styles_num

    def __iter__(self):
        return self

    def __next__(self):
      if self.attr_prob is None:
          for value in self.styles_count.values():
              for i in range(len(value)):
                  value[i] += 1
          self.attr_prob = {key: [] for key in self.styles}
      for value in self.styles_count.values():
          counter = 0
          temp_probs = []
          while counter <= len(value):
              for i in range(len(value)):
                  prob = value[i] / sum([el for el in value])
                  temp_probs.append(prob)
                  counter += 1
              self.attr_prob[list(filter(lambda x: self.styles_count[x] == value, self.styles_count))[0]] = temp_probs
      lengths = [len(lst) for lst in self.styles.values()]
      indices = itertools.product(*[range(l) for l in lengths])
      result = {}
      for index in indices:
          combo = []
          product = 1
          key_combo = []
          for key, lst, i in zip(self.styles.keys(), self.styles.values(), index):
              combo.append(lst[i])
              product *= self.attr_prob[key][i]
              key_combo.append(f"{key}: {lst[i]}")
          result[', '.join(key_combo)] = product
      generated_dict = {}
      generated = choice(list(result.keys()), self.styles_num, p=list(result.values()))
      if self.styles_num == 1:
          generated_dict[generated.item()] = result[generated[0].item()]
          return generated_dict
      else:
          generated_dict = {}
          for i in range(len(generated)):
              generated_dict[generated[i]] = result[generated[i].item()]
          return generated_dict


In [ ]:
style_gen = StyleGenerator(styles, styles_count, styles_num=2)

generated_styles = []
for i in range(2):
    generated_styles.append(next(style_gen))

print(generated_styles)

[{'прическа: длинные прямые, цвет волос: серебристо серый, аксесуар: нет очков, одежда: худи, цвет одежды: зеленый': 0.0008272088091240359, 'прическа: нет волос, цвет волос: пастельный розовый, аксесуар: круглые очки, одежда: комбинезон, цвет одежды: синий': 0.00019282256622937897}, {'прическа: нет волос, цвет волос: каштановый, аксесуар: солнцезащитные очки, одежда: комбинезон, цвет одежды: розовый': 0.00017675401904359734, 'прическа: длинные волнистые, цвет волос: блонд, аксесуар: круглые очки, одежда: худи, цвет одежды: черный': 4.820564155734474e-05}]


#### Generation of pictures

In [2]:
%%capture
!mkdir -p /content/avatars
%cd /content/avatars

In [3]:
files.upload()

Saving avataaars (4).png to avataaars (4).png
Saving avataaars (5).png to avataaars (5).png
Saving avataaars (6).png to avataaars (6).png
Saving avataaars (7).png to avataaars (7).png
Saving avataaars (8).png to avataaars (8).png
Saving avataaars (9).png to avataaars (9).png
Saving avataaars (10).png to avataaars (10).png
Saving avataaars (11).png to avataaars (11).png
Saving avataaars (12).png to avataaars (12).png
Saving avataaars (13).png to avataaars (13).png
Saving avataaars (14).png to avataaars (14).png


{'avataaars (4).png': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x10\x00\x00\x020\x08\x06\x00\x00\x00\xd7\xacR\xbf\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00 \x00IDATx^\xec\x9d\x07\x98T\xd5\xf9\xc6\xdf\xe9m{\xefK\x17A\x14"\x96\xd8\x90\x8e`\x89\xdd\xc4\xd8\x12\x0b\x88 `\xcd?F\xa3)Fc\xa1X\xb0\xc5h\xd4\x88FcE\x91\xa6\xa2\xc6\x82\x8aT\x91\xba}\xd9\xbe;;\xbd\xfd\x9f3\xb8\xb8\xc0\xc2\xcc\x9d\xb9\xd3\xdf\xfb<\xfb,:\xe7|\xe5\xf7\x9d\x9dy\xe7\xdeS\x14\xe0E\x02$@\x02$@\x02$@\x02\x12\t($\xb6gs\x12 \x01\x12 \x01\x12 \x01\x12\x00\x05\x04\x07\x01\t\x90\x00\t\x90\x00\t\x90\x80d\x02\x14\x10\x92\x91\xb1\x03\t\x90\x00\t\x90\x00\t\x90\x00\x05\x04\xc7\x00\t\x90\x00\t\x90\x00\t\x90\x80d\x02\x14\x10\x92\x91\xb1\x03\t\x90\x00\t\x90\x00\t\x90\x00\x05\x04\xc7\x00\t\x90\x00\t\x90\x00\t\x90\x80d\x02\x14\x10\x92\x91\xb1\x03\t\x90\x00\t\x90\x00\t\x90\x00\x05\x04\xc7\x00\t\x90\x00\t\x90\x00\t\x90\x80d\x02\x14\x10\x92\x91\xb1\x03\t\x90\x00\t\x90\x00\t\x90\x00\x05\x04\xc7\x00\t\x90\x00\t\x90\x00\t\x90\x80

In [4]:
image_folder = '/content/avatars'
image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]
images = {}
for f in image_files:
    img = Image.open(os.path.join(image_folder, f))
    if img.mode != 'RGB':
      img = img.convert('RGB')
    if f == 'avataaars (10).png':
      print(img.size)
    images[f] = np.array(img)

(528, 560)


In [ ]:
images.items()

dict_items([('avataaars (10).png', array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)), ('avataaars (9).png', array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 

In [ ]:
pixel_counts = {}
for name, image in images.items():
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            R, G, B = image[i, j]
            red_value = 1 + R
            green_value = 1 + G
            blue_value = 1 + B
            pixel_counts[f"pixel{i}"] = (red_value, green_value, blue_value)


In [ ]:
red_values = [pixel[1][0] for pixel in pixel_counts.items()]
green_values = [pixel[1][1] for pixel in pixel_counts.items()]
blue_values = [pixel[1][2] for pixel in pixel_counts.items()]

red_dist = Counter(red_values)
green_dist = Counter(green_values)
blue_dist = Counter(blue_values)

In [ ]:
blue_dist

Counter({1: 560})

In [ ]:
def generate_value(dist):
    total = sum(dist.values())
    r = random.randint(1, total)
    for value, count in dist.items():
        r -= count
        if r <= 0:
            return value


In [ ]:
new_r = generate_value(red_dist)
new_g = generate_value(green_dist)
new_b = generate_value(blue_dist)

new_pixel = (new_r, new_g, new_b)
new_pixel

In [ ]:
img = Image.new('RGB', (560, 528))

for x in range(560):
    for y in range(528):
        new_r = generate_value(red_dist)
        new_g = generate_value(green_dist)
        new_b = generate_value(blue_dist)
        new_pixel = (new_r, new_g, new_b)
        img.putpixel((x, y), new_pixel)

img.save('new_image.png')

In [5]:
class ImageGenerator_mle:

    def __init__(self, image_folder):
        self.image_folder = image_folder
        self.images = self.load_images()

    def load_images(self):
        images = []
        for filename in os.listdir(self.image_folder):
            if filename.endswith('.png'):
                img = Image.open(os.path.join(self.image_folder, filename))
                images.append(np.array(img))
        return images

    def generate_image(self):
        base_image = random.choice(self.images)
        new_image = np.zeros_like(base_image)
        for i in range(base_image.shape[0]):
            for j in range(base_image.shape[1]):
                pixel_values = [img[i, j] for img in self.images]
                new_image[i, j] = random.choice(pixel_values)
        return Image.fromarray(new_image)

In [6]:
%cd ..
image_gen = ImageGenerator_mle('/content/avatars')
!mkdir mle_avatars
%cd ./mle_avatars
for i in range(5):
    image = image_gen.generate_image()
    image.save(f'mle_avatar_{i}.png')

/content
/content/mle_avatars


In [7]:
class ImageGenerator_poly:

    def __init__(self, image_folder):
        self.image_folder = image_folder
        self.images = self.load_images()

    def load_images(self):
        images = []
        for filename in os.listdir(self.image_folder):
            if filename.endswith('.png'):
                img = Image.open(os.path.join(self.image_folder, filename))
                images.append(np.array(img))
        return images

    def generate_image(self):
        base_image = random.choice(self.images)
        new_image = np.zeros_like(base_image)
        for i in range(base_image.shape[0]):
            for j in range(base_image.shape[1]):
                pixel_values = [img[i, j] for img in self.images]
                unique, counts = np.unique(pixel_values, return_counts=True)
                probabilities = counts / counts.sum()
                new_image[i, j] = np.random.choice(unique, p=probabilities)
        return Image.fromarray(new_image)

In [9]:
%cd ..
image_gen = ImageGenerator_poly('/content/avatars')
!mkdir poly_avatars
%cd ./poly_avatars
for i in range(5):
    image = image_gen.generate_image()
    image.save(f'poly_avatar_{i}.png')

/content
/content/poly_avatars


## Part 2

In [11]:
files.upload()

Saving dataset.zip to dataset.zip


In [ ]:
!unzip /content/dataset.zip

In [ ]:
class Encoder(nn.Module):
    def __init__(self, latent_dims):
        super(Encoder, self).__init__()
        self.linear1 = nn.Linear(784, 512)
        self.linear2 = nn.Linear(512, latent_dims)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.linear1(x))
        return self.linear2(x)

class Decoder(nn.Module):
    def __init__(self, latent_dims):
        super(Decoder, self).__init__()
        self.linear1 = nn.Linear(latent_dims, 512)
        self.linear2 = nn.Linear(512, 784)

    def forward(self, z):
        z = F.relu(self.linear1(z))
        z = torch.sigmoid(self.linear2(z))
        return z.reshape((-1, 1, 28, 28))

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(latent_dims)
        self.decoder = Decoder(latent_dims)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

In [ ]:
class ImageData(torch.utils.data.Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.image_files = os.listdir(img_dir)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.image_files[idx])
        image = torchvision.io.read_image(img_path).float() / 255.0
        if self.transform:
            image = self.transform(image)
        return image

In [ ]:
def train(autoencoder, data, epochs=20):
    opt = torch.optim.Adam(autoencoder.parameters())
    for epoch in tqdm.tqdm(range(epochs)):
        for x in data:
            x = x.to(device)
            opt.zero_grad()
            x_hat = autoencoder(x)
            loss = ((x - x_hat)**2).sum()
            loss.backward()
            opt.step()
    return autoencoder

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
latent_dims = 2
autoencoder = Autoencoder(latent_dims).to(device)

transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((28, 28)),
    torchvision.transforms.Grayscale(),
])

train_data = ImageData('/content/dataset/train', transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
autoencoder = train(autoencoder, train_loader)

In [ ]:
prolivy = ImageData('content/dataset/proliv', transform=transform)
prolivy_loader = torch.utils.data.DataLoader(prolivy, batch_size=128, shuffle=True)

reconstruction_errors_train = []
for img in train_loader:
    img = img.to(device)
    with torch.no_grad():
        reconstruction = autoencoder(img)
    error = F.mse_loss(reconstruction, img, reduction='none').reshape(img.shape[0], -1).mean(1)
    reconstruction_errors_train.append(error)
reconstruction_errors_train = torch.cat(reconstruction_errors_train)

threshold_train = reconstruction_errors_train.mean()
print(threshold_train)

In [ ]:
reconstruction_errors_prolivy = []
for img in prolivy_loader:
    img = img.to(device)
    with torch.no_grad():
        reconstruction = autoencoder(img)
    error = F.mse_loss(reconstruction, img, reduction='none').reshape(img.shape[0], -1).mean(1)
    reconstruction_errors_prolivy.append(error)
reconstruction_errors_proliv = torch.cat(reconstruction_errors_prolivy)

threshold_prolivy = reconstruction_errors_prolivy.mean()
print(threshold_prolivy)

In [ ]:
mean_error = torch.mean(reconstruction_errors_train)
std_error = torch.std(reconstruction_errors_train)

num_std_above_mean = 3

threshold = mean_error + num_std_above_mean * std_error
print(threshold)

In [ ]:
def prolivy_classify(img):
    img = img.to(device)
    with torch.no_grad():
        reconstruction = autoencoder(img)
    error = F.mse_loss(reconstruction, img, reduction='none').reshape(img.shape[0], -1).mean(1)
    return error > threshold

In [ ]:
test_data = ImageData('/content/dataset/test/imgs', transform=transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=True)
with open('/content/dataset/test/test_annotation.txt', 'r') as f:
    annotations = f.readlines()
annotations = {line.split()[0]: int(line.split()[1]) for line in annotations}

In [ ]:
TP, TN, FP, FN = 0, 0, 0, 0

for img, filename in zip(test_loader, test_data.image_files):
    predictions = prolivy_classify(img)
    actual = annotations[filename]
    for prediction in predictions:
        if prediction.item() and actual:
            TP += 1
        elif not prediction.item() and not actual:
            TN += 1
        elif prediction.item() and not actual:
            FP += 1
        elif not prediction.item() and actual:
            FN += 1

TPR = TP / (TP + FN)
TNR = TN / (TN + FP)
acc = (TP + TN) / (TP + TN + FP + FN)


print(f'True Positive Rate: {TPR}')
print(f'True Negative Rate: {TNR}')
print(f'Accuracy: {acc}')